# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [1]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

/Users/jamesrushing/opt/miniconda3/envs/morph/lib/python3.7/site-packages/pyscf/lib/misc.py:50: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
gsd_file = "/Users/jamesrushing/cmelab/data/workspace/d753f6a7bda4d15489a5f38c1e24a17e/trajectory.gsd"

with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[0]
    end_snap = f[-1]

    

ref_distance = 3.563594872561358

start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance

box = start_snap.configuration.box[:3]
unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
unwrap_snap = deepcopy(start_snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1)
comp.visualize().show()

#box = end_snap.configuration.box[:3]
#unwrapped_positions = end_snap.particles.position + end_snap.particles.image * box
#unwrap_snap = deepcopy(end_snap)
#unwrap_snap.particles.position = unwrapped_positions
#unwrap_snap.particles.types = [amber_dict[i].symbol for i in end_snap.particles.types]
#comp = from_snapshot(unwrap_snap, scale=0.1)
#comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
print(start_snap.configuration.box)
print(end_snap.configuration.box)

[389.9132 389.9132 389.9132   0.       0.       0.    ]
[78.29458 78.29458 78.29458  0.       0.       0.     ]


below is some scratch for breaking chromophores into pieces of smiles strings. may be useful down the road but for now its easier to explicity prescibe atom indeces for your chromo

In [4]:
#smarts_str = "c1ccccc1"
#aaids=[]
#aaids_pent = chromophores.get_chromo_ids_smiles(snap, "c1sc2ccsc2c1", amber_dict)
#comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)

#aaids_benzene = chromophores.get_chromo_ids_smiles(snap, smarts_str, amber_dict)
#aaids_thiothene = chromophores.get_chromo_ids_smiles(snap, "c1sc(C)cc1", amber_dict)
#aaids.extend(aaids_benzene)
#aaids.extend(aaids_thiothene)
#aaids.extend(aaids_pent)


the indexes below came from viewing the single molecule trajectory in vmd and clicking on the molecules that I like for my chromos. this cell visualizes those chosen

In [5]:
chromo_ids = np.array([0,1,2,4,6,7,10,11,12,13,19,20,22,23,24,25,27,28,29,30,31,32,91,92,93,94,97,98,99,100,101,102,161,162,163,165,166,168,169,170,171,172,178,179,180,181,177,175,176,174,173,17,15,18,16,14,13])
for i,p in enumerate(comp.particles()):
    if i in chromo_ids:
        p.name = "Kr"
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Trying to get homo and lumo for a single molecule of itic below. using functions in execute_qcc.py. here we use all molecule ids from start_snap.particles.N to get homo lumos for sinlge molecule gsd

displayed : 

HOMO-1, HOMO, LUMO, LUMO+1 above. 

In [6]:
ids = np.arange(start_snap.particles.N)
qcc_input = eqcc.write_qcc_inp(start_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


HOMO-1, HOMO, LUMO, LUMO+1
[-7.22455996 -7.22331261 -0.84719954 -0.84598309]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
ids = np.arange(start_snap.particles.N)
qcc_input = eqcc.write_qcc_inp(end_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)

HOMO-1, HOMO, LUMO, LUMO+1
[-6.46387468 -6.12408723 -1.72460141 -1.70668981]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
qcc_input = eqcc.write_qcc_inp(start_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


/Users/jamesrushing/opt/miniconda3/envs/morph/lib/python3.7/site-packages/pyscf/semiempirical/mindo3.py:137: RuntimeWarning: divide by zero encountered in true_divide
  mopac_param.E2/distances_in_AA - gamma)
/Users/jamesrushing/opt/miniconda3/envs/morph/lib/python3.7/site-packages/pyscf/scf/hf.py:181: RuntimeWarning: invalid value encountered in double_scalars
  cycle+1, e_tot, e_tot-last_hf_e, norm_gorb, norm_ddm)
/Users/jamesrushing/opt/miniconda3/envs/morph/lib/python3.7/site-packages/pyscf/scf/hf.py:185: RuntimeWarning: invalid value encountered in double_scalars
  elif abs(e_tot-last_hf_e) < conv_tol and norm_gorb < conv_tol_grad:


HOMO-1, HOMO, LUMO, LUMO+1
[-6.1520147  -2.03998076 -0.54430535 -0.13805716]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

below i want to get the homolumos for just that back bone as taken from a single molecule gsd

In [9]:
qcc_input = eqcc.write_qcc_inp(end_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


HOMO-1, HOMO, LUMO, LUMO+1
[-5.65292808 -2.31210024 -1.52374795 -0.98395494]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

above this is just the single molecule homo lumo calcs. below i want to bring the build created gsd and get pairwise calcs. the mbuild stuff is being weird . going to just use a two molecule gsd from plankton to begin

In [6]:
chromo2_ids = chromo_ids +186

In [7]:
aaids=[]

In [8]:
aaids.append(chromo_ids)
aaids.append(chromo2_ids)

In [9]:
print(aaids)

[array([  0,   1,   2,   4,   6,   7,  10,  11,  12,  13,  19,  20,  22,
        23,  24,  25,  27,  28,  29,  30,  31,  32,  91,  92,  93,  94,
        97,  98,  99, 100, 101, 102, 161, 162, 163, 165, 166, 168, 169,
       170, 171, 172, 178, 179, 180, 181, 177, 175, 176, 174, 173,  17,
        15,  18,  16,  14,  13]), array([186, 187, 188, 190, 192, 193, 196, 197, 198, 199, 205, 206, 208,
       209, 210, 211, 213, 214, 215, 216, 217, 218, 277, 278, 279, 280,
       283, 284, 285, 286, 287, 288, 347, 348, 349, 351, 352, 354, 355,
       356, 357, 358, 364, 365, 366, 367, 363, 361, 362, 360, 359, 203,
       201, 204, 202, 200, 199])]


In [10]:
chromo_list = []
for i,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(i, start_snap, aaid, "acceptor", amber_dict))

In [11]:
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, end_snap, amber_dict, d_cut=100)
print(f"There are {len(qcc_pairs)} chromophore pairs")

There are 1 chromophore pairs


In [12]:
print(qcc_pairs)

[((0, 1), 'C -5.625373229158573 35.875719455624356 -0.21744091583271086; C -5.1271966544515415 36.735042003536954 -1.229793859253121; C -4.30461822427576 36.2180942572601 -2.2377951867677695; C -3.782454833162479 34.937123683834805 -2.048521352539254; C -4.213975295198612 34.06383982076718 -1.0210965402223593; C -5.205228194368534 34.521723655606046 -0.1332810647584921; C -3.2257569877034946 33.04873981847836 -0.6306527383424765; C -2.0659993735677524 33.67837896719052 -1.3198235757448202; C -2.354781493318729 34.90664425268246 -2.036542249450875; O -1.5918868628866 35.752611545468106 -2.5041154153444296; C -1.1029084769613071 33.17040100469662 -2.1065152413942343; C -0.7985719291097446 31.83830109014584 -2.413777662048531; S 0.7338425072305874 31.870528606320157 -1.8479216821290976; C -0.030228003633670397 31.6011349238403 -0.2511666543581015; C -0.8551515189534946 30.59623875989987 -0.43506176544208586; C -1.3675225821859165 30.77318706884457 -1.7811930902101523; C 0.0775276573770717

In [13]:
i =  0# try any number from 0 to 180
print(f"Pair #{i}:")
visualize_qcc_input(qcc_pairs[i][1])

Pair #0:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
%%time
dimer_data = eqcc.dimer_homolumo(qcc_pairs, "two-molecule-test.txt")

CPU times: user 23.9 ms, sys: 46.6 ms, total: 70.4 ms
Wall time: 17.8 s


In [15]:
%%time
data = eqcc.singles_homolumo(chromo_list, "singles.txt")

CPU times: user 13.6 ms, sys: 32.6 ms, total: 46.2 ms
Wall time: 6.32 s


In [16]:
%%time
eqcc.set_energyvalues(chromo_list, "singles.txt", "two-molecule-test.txt")

CPU times: user 957 µs, sys: 811 µs, total: 1.77 ms
Wall time: 1.31 ms


In [17]:
print(chromo_list)

[Chromophore 0 (acceptor): 57 atoms at -60.002 23.006 88.592, Chromophore 1 (acceptor): 57 atoms at -75.440 89.268 77.741]


In [18]:
i = 0
chromo = chromo_list[i]
print(f"Chromophore {i}:")
print(f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} LUMO+1: {chromo.lumo_1:.2f}")
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

Chromophore 0:
HOMO-1: -6.15 HOMO: -2.04 LUMO: -0.54 LUMO+1: -0.14
1 neighbors
DeltaE of first neighbor: 0.000
Transfer integral of first neighbor: 0.512
